In [1]:
import wandb
import numpy as np
import torch
import lightning
import copy
from pathlib import Path
from model_fusion.config import BASE_DATA_DIR, CHECKPOINT_DIR
from model_fusion.datasets import DataModuleType
from model_fusion.models import ModelType
from model_fusion.models.lightning import BaseModel 
from Experiments import lmc_experiment
from model_fusion import lmc_utils
from Experiments import baselines_experiment
from Experiments import otfusion_experiment
from Experiments import pyhessian_experiment
from model_fusion.train import setup_training, setup_testing


# set seed for numpy based calculations
NUMPY_SEED = 100
np.random.seed(NUMPY_SEED)

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
c:\Users\filos\OneDrive\Desk

In [2]:
print("------- Loading models -------")

# select wandb run names
runA = '33kyx0p1'
runB = '5tbujwji'#diff init

api = wandb.Api()
run = api.run(f'model-fusion/Model Fusion/{runA}')

print(run.config)

batch_size = run.config['datamodule_hparams'].get('batch_size')

datamodule_type_str = run.config['datamodule_type'].split('.')[1].lower()
datamodule_type = DataModuleType(datamodule_type_str)
datamodule_hparams = run.config['datamodule_hparams']
datamodule_hparams['data_augmentation'] = False

model_type_str = run.config['model_type'].split('.')[1].lower()
model_type = ModelType(model_type_str)

model_hparams = run.config['model_hparams']

print(datamodule_hparams)
print(model_hparams)

checkpointA = f'model-fusion/Model Fusion/model-{runA}:best_k'
checkpointB = f'model-fusion/Model Fusion/model-{runB}:best_k'

run = wandb.init()

artifact = run.use_artifact(checkpointA, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelA = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

artifact = run.use_artifact(checkpointB, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelB = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")


------- Loading models -------


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'lr': 0.1, 'model': None, 'momentum': 0.9, 'data_seed': 42, 'optimizer': 'sgd', 'max_epochs': 200, 'min_epochs': 50, 'model_seed': 42, 'model_type': 'ModelType.VGG11', 'loss_module': 'CrossEntropyLoss', 'lr_scheduler': 'plateau', 'weight_decay': 0.0001, 'model_hparams': {'bias': False, 'num_classes': 10, 'num_channels': 3}, 'early_stopping': True, 'datamodule_type': 'DataModuleType.CIFAR10', 'lr_decay_factor': 0.1, 'lightning_params': {'lr': 0.1, 'momentum': 0.9, 'data_seed': 42, 'optimizer': 'sgd', 'model_seed': 42, 'lr_scheduler': 'plateau', 'weight_decay': 0.0001, 'lr_decay_factor': 0.1, 'lr_monitor_metric': 'val_loss'}, 'lr_monitor_metric': 'val_loss', 'datamodule_hparams': {'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': True}, 'model_hparams/bias': False, 'model_hparams/num_classes': 10, 'model_hparams/num_channels': 3}
{'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': False}
{'bias': False, 'num_classes': 10, 'num_channels': 3}


wandb: Currently logged in as: frambelli (model-fusion). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-33kyx0p1:best_k, 982.65MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:5.7
wandb: Downloading large artifact model-5tbujwji:best_k, 982.65MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.8


In [4]:
# LMC barrier
print("------- Computing LMC barrier before alignment-------")

lmc_experiment.run_lmc(
    datamodule_type=datamodule_type,
    modelA=modelA,
    modelB=modelB,
    granularity=21
)

Seed set to 42


------- Computing LMC barrier before alignment-------
Files already downloaded and verified
Files already downloaded and verified


Seed set to 42
Seed set to 42


Alpha: 0.00 (model 2), Train average loss: 0.14353 Train barrier:  0
Alpha: 1.00 (model 1), Train average loss: 0.08886 Train barrier:  0


Seed set to 42


...

Alpha: 0.80, Train average loss: 1.67080 Train barrier 1.5710085857582092


Seed set to 42


Alpha: 0.85, Train average loss: 1.04736 Train barrier 0.9503028320607874


Seed set to 42


Alpha: 0.90, Train average loss: 0.46962 Train barrier 0.37529936182525425
Alpha: 0.95, Train average loss: 0.16947 Train barrier 0.07787863567153613
Loss model 1: 0.08886, Loss model 2: 0.14353, Alpha argmax: 0.55000
Barrier: 2.18875


In [3]:
# Baselines (prediction ensembling, vanilla averaging)
print("------- Computing baselines -------")

wandb_tag = f'baselines-{runA}-{runB}'

vanilla_averaging_model = baselines_experiment.run_baselines(
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
)

------- Computing baselines -------
------- Prediction based ensembling -------
------- Naive ensembling of weights -------
------- Evaluating baselines -------


c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pytorch_lightning\loggers\wandb.py:395: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


------- Evaluating base models -------
Testing DataLoader 0: 100%|██████████| 10/10 [00:02<00:00,  3.35it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8693000078201294
        val_loss            0.5114084482192993
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 10/10 [00:01<00:00,  7.18it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8628999590873718
        val_loss            0.4932338297367096
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
------- Evaluating prediction ensembling -------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Test set: Average loss: 0.4309, Accuracy: 88.29%
------- Evaluating vanilla averaging -------
Testing DataLoader 0: 100%|██████████| 10/10 [00:01<00:00,  5.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.2206999957561493
        val_loss            2.3025741577148438
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁
trainer/global_step,▁▁▁
val_accuracy,██▁
val_loss,▁▁█
epoch,0
trainer/global_step,0
val_accuracy,0.2207
val_loss,2.30257


In [3]:
# OT model fusion + eval aligned model 
print("------- Computing model fusion -------")

wandb_tag = f"ot_model_fusion-{runA}-{runB}"

ot_fused_model, modelA_aligned = otfusion_experiment.run_otfusion(
    batch_size=batch_size,
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
    is_vgg=True
)


Seed set to 42


------- Computing model fusion -------
------- Setting up parameters -------
{'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': False}
The parameters are: 
 {'eval_aligned': True, 'num_models': 2, 'width_ratio': 1, 'handle_skips': False, 'exact': True, 'activation_seed': 21, 'activation_histograms': True, 'ground_metric': 'euclidean', 'ground_metric_normalize': 'none', 'same_model': False, 'geom_ensemble_type': 'acts', 'act_num_samples': 75, 'skip_last_layer': False, 'skip_last_layer_type': 'average', 'softmax_temperature': 1, 'past_correction': True, 'correction': True, 'normalize_acts': False, 'normalize_wts': False, 'activation_normalize': False, 'center_acts': False, 'prelu_acts': False, 'pool_acts': False, 'pool_relu': False, 'importance': None, 'proper_marginals': False, 'not_squared': True, 'dist_normalize': False, 'clip_gm': False, 'clip_min': 0, 'clip_max': 5, 'tmap_stats': False, 'ensemble_step': 0.5, 'ground_metric_eff': False, 'reg': 0.01}
------- OT m

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\ot\lp\__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


At layer idx 8 and shape torch.Size([4096, 25088]), the OT cost is  2.048359497908109
Tmap stats (before correction) 
: For layer model.classifier.0.weight, frobenius norm from the joe's transport map is 0.015623092651367188
shape of T_var is  torch.Size([4096, 4096])
T_var before correction  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
T_var after correction  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
T_var stats: max 0.9995905756950378, min 0.0, mean 0.00024404068244621158, std 0.015616696327924728 
the transport map is  tensor([[0., 0., 

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pytorch_lightning\loggers\wandb.py:395: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 20/20 [00:01<00:00, 12.44it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.7537999749183655
        val_loss            1.8387837409973145
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
trainer/global_step,▁
val_accuracy,▁
val_loss,▁
epoch,0
trainer/global_step,0
val_accuracy,0.7538
val_loss,1.83878


In [7]:
save_path = r'C:\Users\filos\OneDrive\Desktop\ETH\model-fusion\VGG_cifar10_models'
modelOTF_name = 'modelOTF_512AC_VGG.t7'
torch.save(ot_fused_model.state_dict(), save_path + '\\' + modelOTF_name)

In [8]:
# LMC barrier
print("------- Computing LMC barrier after alignment -------")

lmc_experiment.run_lmc(
    datamodule_type=datamodule_type,
    modelA=modelA_aligned,
    modelB=modelB,
    granularity=21
)

# Losses for ot fusion model and vanilla averaging model
datamodule_hparams_lmc = {'batch_size': 1024, 'data_dir': BASE_DATA_DIR}
datamodule_lmc = datamodule_type.get_data_module(**datamodule_hparams)
datamodule_lmc.prepare_data()
datamodule_lmc.setup('fit')

vanilla_loss = lmc_utils.compute_loss(vanilla_averaging_model, datamodule_lmc)
fused_loss = lmc_utils.compute_loss(ot_fused_model, datamodule_lmc)

print(f"Vanilla loss pre fine-tuning: {vanilla_loss}")
print(f"Fused loss pre fine-tuning: {fused_loss}")

Seed set to 42


------- Computing LMC barrier after alignment -------
Files already downloaded and verified
Files already downloaded and verified


Seed set to 42
Seed set to 42


Alpha: 0.00 (model 2), Train average loss: 0.14353 Train barrier:  0


Seed set to 42


...

Alpha: 0.85, Train average loss: 0.46804 Train barrier 0.37138481511433924


Seed set to 42


Alpha: 0.90, Train average loss: 0.25285 Train barrier 0.15894587491459317
Alpha: 0.95, Train average loss: 0.13166 Train barrier 0.040511892658339585
Loss model 1: 0.08839, Loss model 2: 0.14353, Alpha argmax: 0.50000
Barrier: 1.70986
Files already downloaded and verified
Files already downloaded and verified


Seed set to 42
Seed set to 42


Vanilla loss pre fine-tuning: 2.3025731369018554
Fused loss pre fine-tuning: 1.8252799756368001


In [4]:
# finetuning otfused model   
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

from model_fusion.train import get_wandb_logger

_, datamodule, trainer = None, None, None

min_epochs = 50
max_epochs = 100
datamodule_hparams['batch_size'] = 512
datamodule_hparams['data_augmentation']=True

datamodule = datamodule_type.get_data_module(**datamodule_hparams)
lightning_params = {'optimizer': 'sgd', 'lr': 0.1, 'momentum': 0.95, 'weight_decay': 0.0001, 'lr_scheduler': 'plateau', 'lr_decay_factor': 0.5, 'lr_monitor_metric': 'val_loss'}
otfused_lit_model = BaseModel(model_type=model_type, model_hparams=model_hparams, model=copy.deepcopy(ot_fused_model.model), **lightning_params)


logger_config = {'model_hparams': model_hparams} | {'datamodule_hparams': datamodule_hparams} | {'lightning_params': lightning_params} | {'min_epochs': min_epochs, 'max_epochs': max_epochs, 'model_type': model_type, 'datamodule_type': datamodule_type, 'early_stopping': True}
logger = get_wandb_logger("otfusion finetuning", logger_config, [])
callbacks = []
monitor = 'val_loss'
patience = 20
callbacks.append(EarlyStopping(monitor=monitor, patience=patience))

checkpoint_callback = ModelCheckpoint(monitor="val_accuracy", mode="max")
callbacks.append(checkpoint_callback)
trainer = lightning.Trainer(min_epochs=min_epochs, max_epochs=max_epochs, logger=logger, callbacks=callbacks, deterministic='warn')


datamodule.prepare_data()

datamodule.setup('fit')

trainer.fit(otfused_lit_model, train_dataloaders=datamodule.train_dataloader(), val_dataloaders=datamodule.val_dataloader())


datamodule.setup('test')
trainer.test(otfused_lit_model, dataloaders=datamodule.test_dataloader())


wandb.finish()

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | loss_module | CrossEntropyLoss   | 0     
1 | accuracy    | MulticlassAccuracy | 0     
2 | model       | VGG                | 128 M 
---------------------------------------------------
128 M     Trainable params
0         Non-trainable params
128 M     Total params
515.185   Total estimated model params size (MB)


Using SGD optimizer with lr=0.1, momentum=0.95, weight_decay=0.0001, nesterov=False
Using ReduceLROnPlateau with lr_decay_factor=0.5 and lr_monitor_metric=val_loss
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:156.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but

Epoch 66: 100%|██████████| 88/88 [00:25<00:00,  3.48it/s, v_num=dgmw, train_loss=0.0518, train_accuracy=0.985, val_loss=0.570, val_accuracy=0.893, avg_train_loss=0.0351] 


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 20/20 [00:02<00:00,  8.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8844000101089478
        val_loss            0.6146135330200195
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


avg_train_loss,█▇▆▆▆▆▆▆▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▃▃▅▂▄▄▄▅▅▅▆▆▆▆▆▆▆▇▆▆▇▆▇▆▇▇▇▇▇▇█████████
train_loss,█▇▆▅▇▅▆▆▅▄▃▄▃▃▃▃▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▁▂▁▂▁▃▅▅▅▅▆▆▆▆▆▆▇▆▆▆▆▆▆▆▇▇▇▇▇▇▇████████
val_loss,█▇▇██▇▇▄▄▄▃▂▂▂▃▃▂▂▂▂▂▃▂▂▃▂▂▁▁▂▂▂▂▂▂▃▂▂▃▄
avg_train_loss,0.03516
epoch,67
train_accuracy,0.99023
train_loss,0.02733


In [6]:
#Testing ot fused model finetuned
runFT = 'oq06dgmw'

api = wandb.Api()
run = api.run(f'model-fusion/Model Fusion/{runFT}')

print(run.config)

batch_size = run.config['datamodule_hparams'].get('batch_size')

datamodule_type_str = run.config['datamodule_type'].split('.')[1].lower()
datamodule_type = DataModuleType(datamodule_type_str)
datamodule_hparams = run.config['datamodule_hparams']
datamodule_hparams['data_augmentation'] = False

model_type_str = run.config['model_type'].split('.')[1].lower()
model_type = ModelType(model_type_str)

model_hparams = run.config['model_hparams']

print(datamodule_hparams)
print(model_hparams)

checkpointFT = f'model-fusion/Model Fusion/model-{runFT}:v18'

run = wandb.init()

artifact = run.use_artifact(checkpointFT, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
otfused_lit_model = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")
wandb_tags = [f"{model_type.value}", f"{datamodule_type.value}"]

datamodule, trainer = setup_testing(f'eval finetuning {runFT}', model_type, model_hparams, datamodule_type, datamodule_hparams, wandb_tags)


datamodule.prepare_data()
datamodule.setup('test')

trainer.test(otfused_lit_model, dataloaders=datamodule.test_dataloader())

wandb.finish()

finetuned_loss = lmc_utils.compute_loss(otfused_lit_model, datamodule_lmc)

print(f"Finetuned otfused loss: {finetuned_loss}")

{'lr': 0.1, 'model': None, 'momentum': 0.95, 'optimizer': 'sgd', 'max_epochs': 100, 'min_epochs': 50, 'model_type': 'ModelType.VGG11', 'loss_module': 'CrossEntropyLoss', 'lr_scheduler': 'plateau', 'weight_decay': 0.0001, 'model_hparams': {'bias': False, 'num_classes': 10, 'num_channels': 3}, 'early_stopping': True, 'datamodule_type': 'DataModuleType.CIFAR10', 'lr_decay_factor': 0.5, 'lightning_params': {'lr': 0.1, 'momentum': 0.95, 'optimizer': 'sgd', 'lr_scheduler': 'plateau', 'weight_decay': 0.0001, 'lr_decay_factor': 0.5, 'lr_monitor_metric': 'val_loss'}, 'lr_monitor_metric': 'val_loss', 'datamodule_hparams': {'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': True}, 'model_hparams/bias': False, 'model_hparams/num_classes': 10, 'model_hparams/num_channels': 3}
{'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': False}
{'bias': False, 'num_classes': 10, 'num_channels': 3}


wandb: Downloading large artifact model-oq06dgmw:v18, 982.66MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.1
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pytorch_lightning\loggers\wandb.py:395: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 20/20 [00:01<00:00, 12.38it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8794999718666077
        val_loss            0.5817849040031433
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
trainer/global_step,▁
val_accuracy,▁
val_loss,▁
epoch,0
trainer/global_step,0
val_accuracy,0.8795
val_loss,0.58178


Seed set to 42


Finetuned otfused loss: 0.03635795477496253


In [12]:
# Pyhessian (compute sharpness and eigenspectrum of base models, vanilla avg, ot fusion and finetuned solutions)
print("------- Computing sharpness -------")

print("------- Model A -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=modelA, compute_density=False, figure_name='modelA.pdf') 

print("------- Model B -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=modelB, compute_density=False, figure_name='modelB.pdf')

print("------- Model A aligned to B -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelA_aligned,  compute_density=False, figure_name='modelA_aligned.pdf')

print("------- OT fusion model -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=ot_fused_model,  compute_density=False, figure_name='otmodel_diff_init32_VGG.pdf')

print("------- Vanilla avg model -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=vanilla_averaging_model,  compute_density=False, figure_name='vanilla_avg.pdf')

Seed set to 42


------- Computing sharpness -------
------- Model A -------
Files already downloaded and verified
Files already downloaded and verified


c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ..\torch\csrc\autograd\engine.cpp:1176.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable d

The top Hessian eigenvalue of this model is 6.3733


Seed set to 42


...

Seed set to 42



***Trace:  353.85181971958707
------- OT fusion model -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is 3.3642


Seed set to 42



***Trace:  53.577320684086196
------- Vanilla avg model -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is -0.0023


KeyboardInterrupt: 

In [8]:
print("------- OT fusion model (finetuned) -------")
otfused_lit_model.to('cuda')
print(otfused_lit_model.device)
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=otfused_lit_model,num_batches=30,  compute_density=False, figure_name='otmodel.pdf')

Seed set to 42


------- OT fusion model (finetuned) -------
cuda:0
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is 4.0483

***Trace:  242.58398021351206


In [ ]:
save_path = r'C:\Users\filos\OneDrive\Desktop\ETH\model-fusion\Vgg_cifar10_models'
modelA_name = 'model512A_VGG.t7'
torch.save(modelA.state_dict(), save_path + '\\' + modelA_name)

modelB_name = 'model512C_VGG.t7'
torch.save(modelB.state_dict(), save_path + '\\' + modelB_name)

modelA_aligned_name = 'model512A_aligned_to512C_VGG.t7'
torch.save(modelA_aligned.state_dict(), save_path + '\\' + modelA_aligned_name)